# setup

In [ ]:
import os
from threading import Thread


def bkgw():
  !apt-get install -y libvulkan-dev libomp5
  !pip install ncnn-vulkan
  !mv /usr/local/lib/python3.7/dist-packages/ncnn_vulkan/*.so /usr/local/lib/python3.7/dist-packages/ncnn/
  os.link('/content/vqdec18.param','/content/setupfin.txt')


t3 = Thread(target = bkgw)
a3 = t3.start()
!wget https://github.com/TabuaTambalam/vqqncnn/releases/download/0.2/rcup.zip
!7z e rcup.zip
!wget https://github.com/TabuaTambalam/vqqncnn/releases/download/0.2/rcub.7z
!wget https://github.com/TabuaTambalam/vqqncnn/releases/download/0.0/vq_anime.bin
!mkdir rcubw
!7z e -o/content/rcubw rcub.7z
!rm rcub.7z
!rm rcup.zip

# o

In [ ]:
UseVulkan=False #@param {type:'boolean'}

import gc
import os
import numpy as np
from PIL import Image


class ExtractorGPU:
  def __init__(self,c):
    self.cnet=c
  def __enter__(self):
    self.ex = self.cnet.create_extractor()
    self.ex.set_blob_vkallocator(blob_vkallocator)
    self.ex.set_workspace_vkallocator(blob_vkallocator)
    self.ex.set_staging_vkallocator(staging_vkallocator)
    return self.ex

  def __exit__(self, exc_type, exc_val, exc_tb):
    blob_vkallocator.clear()
    staging_vkallocator.clear()
    self.ex.clear()

class ExtractorCPU:
  def __init__(self,c):
    self.cnet=c
  def __enter__(self):
    self.ex = self.cnet.create_extractor()
    return self.ex

  def __exit__(self, exc_type, exc_val, exc_tb):
    self.ex.clear()

def loaddumps(fna):
  return np.fromfile(fna,dtype=np.uint16).astype(np.uint32).reshape((-1,16,16))

def showp(n):
  uwa=np.pad(dumped_seqs[n],((1,1),(1,1)), 'reflect' )
  with NCNNex(net) as ex:
    ex.input('in0', ncnn.Mat(uwa.reshape(324)).clone())
    hrr, out0 = ex.extract('out0')
  with NCNNex(netrcu) as ex:
    ex.input('in0', out0)
    hrr, out1 = ex.extract('out0')

  ymg=np.array(out1).transpose(1,2,0)
  if upscaleNx == 4:
    krop=np.array(out0).transpose(1,2,0)[16:-16,16:-16,:]
    h,w=ymg.shape[:-1]
    for y in range(h):
      for x in range(w):
        ymg[y,x,:]+=krop[y>>2,x>>2,:]
    

  uz=Image.fromarray( ( ymg*255 ).clip(0,255).astype(np.uint8) )
  uz.save('/content/sample_data/%d.png'%n)
  return uz


ntai='-n%d.bin'
def rculoader():
  global typo
  prm=typo+'.param'
  if not os.path.isfile(prm):
    print('try again, no '+prm)
    return
  if typo.startswith('upn'):
    typo='upn2x'
    nstr='-n0.bin'
  elif typo == 'up2x':
    if Denoise < 5:
      nstr=ntai%Denoise
    else:
      n=Denoise//21
      nstr=ntai%n
  else:
    if Denoise < 3:
      nstr=ntai%Denoise
    else:
      n=Denoise//34
      nstr=ntai%n

  netrcu.load_param(  prm   ) 
  netrcu.load_model('rcubw/'+typo+nstr)


try:
  del net
  gc.collect()
except:
  pass

if os.path.isfile('/content/setupfin.txt'):
  import ncnn
  net = ncnn.Net()
  net.opt.use_vulkan_compute = UseVulkan

  if UseVulkan:
    vkdev = ncnn.get_gpu_device(ncnn.get_default_gpu_index())
    blob_vkallocator = ncnn.VkBlobAllocator(vkdev)
    staging_vkallocator = ncnn.VkStagingAllocator(vkdev)
    NCNNex = ExtractorGPU

  else:
    NCNNex=ExtractorCPU


  net.opt.use_fp16_packed = False
  net.opt.use_fp16_storage = False
  net.opt.use_fp16_arithmetic = False

  net.load_param( 'vqdec18.param'  )
  net.load_model('vq_anime.bin')
else:
  print('wait ncnn setup')

In [ ]:
typo='pro' #@param ['up', 'upn', 'pro']
upscaleNx=3 #@param {type:'integer'}
Denoise=2 #@param {type:'integer'}

if upscaleNx <2:
  if typo=='upn':
    upscaleNx=1
  else:
    upscaleNx=2
elif upscaleNx >4:
  upscaleNx=4
typo+='%dx'%upscaleNx


try:
  del netrcu
  gc.collect()
except:
  pass

netrcu = ncnn.Net()
netrcu.opt.use_vulkan_compute = UseVulkan
rculoader()

In [ ]:
dumped_seqs=loaddumps('/content/Dallesav14.bin')

In [ ]:
showp(10)